## Group feature extraction

In [1]:
import movekit as mkit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path = "./datasets/fish-5-features.csv"
data = mkit.read_data(path)
data.head()

time  animal_id       x       y  distance  average_speed  \
0        1        312  405.29  417.76       0.0       0.210217   
1000     1        511  369.99  428.78       0.0       0.020944   
2000     1        607  390.33  405.89       0.0       0.070235   
3000     1        811  445.15  411.94       0.0       0.370500   
4000     1        905  366.06  451.76       0.0       0.118000   

      average_acceleration  direction  stopped  
0                -0.018039        0.0        1  
1000              0.000236        0.0        1  
2000              0.004961        0.0        1  
3000              0.017482        0.0        1  
4000             -0.006333        0.0        1

### Detecting outliers
Function performs detection of outliers, based on user's criteria, regarding features, method and share of outliers.

In [3]:
outs = mkit.outlier_detection(data)

# printing all rows where outliers are present
outs[outs.loc[:,"outlier"] == 1]

time  animal_id  outlier       x       y  distance  average_speed  \
2479   480        607        1   60.63  401.17  2.418677       2.503849   
2481   482        607        1   60.32  396.22  2.522856       2.479927   
1877   878        511        1  511.80   69.58  4.623505       4.293873   
1878   879        511        1  516.29   70.07  4.516658       4.303061   
1881   882        511        1  528.46   72.19  4.022400       4.136429   
1882   883        511        1  532.33   73.18  3.994621       4.036757   
1982   983        511        1  651.33  170.35  1.787093       1.732611   
1983   984        511        1  651.51  172.52  2.177453       2.115413   
1984   985        511        1  651.65  175.29  2.773536       2.539209   
1985   986        511        1  651.82  178.70  3.414235       2.954890   
1987   988        511        1  652.54  186.89  4.358394       3.577117   
1988   989        511        1  653.07  191.43  4.570831       3.802708   
1989   990        511        1  653.66  195.89  4.498855       3.968672   
1990   991        511        1  654.43  200.05  4.230662       4.037620   
1991   992        511        1  655.48  204.01  4.096840       4.005171   
1992   993        511        1  656.65  207.88  4.042994       3.874063   
1993   994        511        1  657.77  211.55  3.837095       3.637472   

      average_acceleration  direction  stopped  
2479              0.073790 -97.125016        0  
2481              0.056199 -92.726311        0  
1877              0.241837   2.231175        0  
1878              0.187692   6.228122        0  
1881              0.040597  11.617220        0  
1882              0.009188  14.349332        0  
1982              0.158860  80.010593        0  
1983              0.198432  85.258210        0  
1984              0.259326  87.106646        0  
1985              0.327106  87.145975        0  
1987              0.382802  84.734160        0  
1988              0.423797  83.341425        0  
1989              0.415680  82.464268        0  
1990              0.343864  79.513455        0  
1991              0.278364  75.149662        0  
1992              0.225590  73.178590        0  
1993              0.165964  73.029010        0

In [4]:
# same function, different parameters
other_outs = mkit.outlier_detection(dataset = data, features = ["average_speed", "average_acceleration"], contamination = 0.05, n_neighbors = 8, method = "median", metric = "euclidean")

# printing all rows where outliers are present
other_outs[other_outs.loc[:,"outlier"] == 1].head()

time  animal_id  outlier       x       y  distance  average_speed  \
2324   325        607        1  126.90  410.58  1.050000       1.296308   
2325   326        607        1  126.04  410.56  0.860233       1.065177   
2326   327        607        1  125.45  410.65  0.596825       0.876628   
2327   328        607        1  124.93  410.77  0.533667       0.738006   
365    366        312        1  257.86  403.82  2.462458       2.763925   

      average_acceleration   direction  stopped  
2324             -0.224662  180.000000        0  
2325             -0.244628 -178.667780        0  
2326             -0.264200  171.326826        0  
2327             -0.263430  167.005383        0  
365              -0.176182  177.439699        0

### Group-level Analysis

Below we perform Analysis on Group-Level. This consists of:
- Group-Level averages,
- Centroid Medoid computation
- A dynamic time warping matrix, 
- A clustering over time based on absolute features,
- The centroid direction,
- The heading difference of each animal with respect to the current centroid
- The group - polarization for each timestep. 

#### Obtain group-level records for each point in time
Records consist of total group-distance, mean-speed mean-acceleration and mean distance from centroid. If input doesn't contain centroid or feature data, it is calculated, showing a warning.
Parameter object_output produces a post gis - compatible point.

In [5]:
group_data = mkit.group_movement(data)
group_data.head()

c:\users\manuel\appdata\local\programs\python\python38\lib\site-packages\movekit\feature_extraction.py:297: UserWarning: Recalculating centroid-distances, since not found in input dataset!
  warnings.warn(


total_dist  mean_speed  mean_acceleration  mean_distance_centroid
time                                                                   
1       0.000000    0.157979          -0.000339                 29.4616
2       1.174908    0.157641          -0.000339                 29.5850
3       1.025155    0.155610          -0.000339                 29.6914
4       0.918960    0.153579          -0.000339                 29.7782
5       0.830461    0.153341          -0.000339                 29.8518

#### Obtain centroid, medoid and distance to centroid 
Parameter options go either for each unit per timestamp or as post-gis compatible object

In [6]:
movement = mkit.centroid_medoid_computation(data, object_output = False)
movement.head()

time  animal_id  outlier       x       y  distance  average_speed  \
0     1        312        0  405.29  417.76  0.000000       0.210217   
1     2        312        0  405.31  417.37  0.390512       0.192177   
2     3        312        0  405.31  417.07  0.300000       0.174723   
3     4        312        0  405.30  416.86  0.210238       0.159133   
4     5        312        0  405.29  416.71  0.150333       0.155506   

   average_acceleration  direction  stopped  x_centroid  y_centroid  medoid  \
0             -0.018039   0.000000        1     395.364     423.226     312   
1             -0.018039 -87.064327        1     395.382     423.220     312   
2             -0.018039 -90.000000        1     395.392     423.234     312   
3             -0.018039 -92.726311        1     395.396     423.272     312   
4             -0.018039 -93.814075        1     395.394     423.324     312   

   distance_to_centroid  
0                11.331  
1                11.523  
2                11.677  
3                11.798  
4                11.903

#### Get the centroid direction
If no centroid coordinates are found in input, function calculates centroids first, emitting a warning.

In [7]:
centroid_dir = mkit.compute_centroid_direction(data).sort_values(['time','animal_id'])
centroid_dir.head(10)

c:\users\manuel\appdata\local\programs\python\python38\lib\site-packages\movekit\clustering.py:172: UserWarning: x_centroid or y_centroid not found in data. Calculating centroid...
  warnings.warn(


time  animal_id  outlier       x       y  distance  average_speed  \
0        1        312        0  405.29  417.76  0.000000       0.210217   
1000     1        511        0  369.99  428.78  0.000000       0.020944   
2000     1        607        0  390.33  405.89  0.000000       0.070235   
3000     1        811        0  445.15  411.94  0.000000       0.370500   
4000     1        905        0  366.06  451.76  0.000000       0.118000   
1        2        312        0  405.31  417.37  0.390512       0.192177   
1001     2        511        0  370.01  428.82  0.044721       0.021180   
2001     2        607        0  390.25  405.89  0.080000       0.075196   
3001     2        811        0  445.48  412.26  0.459674       0.387983   
4001     2        905        0  365.86  451.76  0.200000       0.111667   

      average_acceleration   direction  stopped  x_centroid  y_centroid  \
0                -0.018039    0.000000        1     395.364     423.226   
1000              0.000236    0.000000        1     395.364     423.226   
2000              0.004961    0.000000        1     395.364     423.226   
3000              0.017482    0.000000        1     395.364     423.226   
4000             -0.006333    0.000000        1     395.364     423.226   
1                -0.018039  -87.064327        1     395.382     423.220   
1001              0.000236   63.434949        1     395.382     423.220   
2001              0.004961  180.000000        1     395.382     423.220   
3001              0.017482   44.118596        1     395.382     423.220   
4001             -0.006333  180.000000        1     395.382     423.220   

      centroid_direction  
0                    NaN  
1000                 NaN  
2000                 NaN  
3000                 NaN  
4000                 NaN  
1             -18.434949  
1001          -18.434949  
2001          -18.434949  
3001          -18.434949  
4001          -18.434949

#### Getting the heading difference
Calculate the difference in degrees between the animal's direction and the centroid's direction for each timestep. Stronger gain in y gives positive difference, weaker gain in y gives negative difference, since constant y is defined to be 0 degrees.

In [8]:
heading_diff = mkit.get_heading_difference(data)
heading_diff.head()

time  animal_id  outlier       x       y  distance  average_speed  \
0     1        312        0  405.29  417.76  0.000000       0.210217   
1     2        312        0  405.31  417.37  0.390512       0.192177   
2     3        312        0  405.31  417.07  0.300000       0.174723   
3     4        312        0  405.30  416.86  0.210238       0.159133   
4     5        312        0  405.29  416.71  0.150333       0.155506   

   average_acceleration  direction  stopped  x_centroid  y_centroid  medoid  \
0             -0.018039   0.000000        1     395.364     423.226     312   
1             -0.018039 -87.064327        1     395.382     423.220     312   
2             -0.018039 -90.000000        1     395.392     423.234     312   
3             -0.018039 -92.726311        1     395.396     423.272     312   
4             -0.018039 -93.814075        1     395.394     423.324     312   

   distance_to_centroid  centroid_direction  heading_difference  
0                11.331                 NaN                 NaN  
1                11.523          -18.434949          -68.629378  
2                11.677           54.462322         -144.462322  
3                11.798           83.990994         -176.717305  
4                11.903           92.202598          173.983327

#### Computing polarization
Computing polarization of animals for each timestep. Value is between 0 and 1

In [9]:
pol = mkit.compute_polarization(data)
pol.head()

time  animal_id  outlier       x       y  distance  average_speed  \
0     1        312        0  405.29  417.76  0.000000       0.210217   
1     2        312        0  405.31  417.37  0.390512       0.192177   
2     3        312        0  405.31  417.07  0.300000       0.174723   
3     4        312        0  405.30  416.86  0.210238       0.159133   
4     5        312        0  405.29  416.71  0.150333       0.155506   

   average_acceleration  direction  stopped  polarization  
0             -0.018039   0.000000        1      1.000000  
1             -0.018039 -87.064327        1      0.248837  
2             -0.018039 -90.000000        1      0.323950  
3             -0.018039 -92.726311        1      0.125276  
4             -0.018039 -93.814075        1      0.037233

#### Obtain a matrix, based on dynamic time warping
Each Animal-ID is displayed in the indices, the entries reflect the similarity of the animal's trajectories based on the DTW algorithm.

In [10]:
mkit.dtw_matrix(data)

312           511           607           811           905
312      0.000000  30843.085403  32859.600139  42461.524553  37916.447829
511  30843.085403      0.000000  26931.014323  47116.708116  20967.960073
607  32859.600139  26931.014323      0.000000  39859.787924  35711.718898
811  42461.524553  47116.708116  39859.787924      0.000000  38379.806433
905  37916.447829  20967.960073  35711.718898  38379.806433      0.000000